In [1]:
import geemap
import ee
Map = geemap.Map()
Map
import pandas as pd

In [2]:
roi = ee.FeatureCollection("users/yehuigeo/poyanghu")

In [3]:
def ND_VI(image,b1,b2,bName):  
    VI = image.normalizedDifference([b1,b2]).rename(bName)
    return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
#这里对大于-1小于1的像元进行掩膜

#定义计算哨兵二号的EVI指数的函数
def funEVI(image,a,b,c):   
    VI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'BLUE': image.select(a),   
      'RED': image.select(b),   
      'NIR': image.select(c)}).rename('EVI')
    return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
#定义添加波段的函数
def addLandsatVIs(img):
    NDVI = ND_VI(img,'B8','B4','NDVI')
    EVI = funEVI(img,'B2','B4','B8')
    mNDWI = ND_VI(img,'B3','B6','mNDWI')
    return img.addBands(NDVI).addBands(EVI).addBands(mNDWI)
    # 水提取 MNDWI > NDVI或MNDWI > EVI 和 EVI < 0.1

In [4]:
collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

In [15]:
for year in range(2013,2021):
    START_DATE = f'{year}'
    END_DATE = f'{year + 1}'
    collection5 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') .filterBounds(roi).filterDate(START_DATE, END_DATE ).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa']).map(addLandsatVIs)   
    #print(collection5.aggregate_array('WRS_PATH').getInfo())
    pa = pd.DataFrame(collection5.aggregate_array('WRS_PATH').getInfo())      # aggregate_array为获取属性的意思  # pd.dataframe为将属性和属性值转换为dataframe的意思
    #print(pa)
    path1 = path * 100                
    row = pd.DataFrame(collection5.aggregate_array('WRS_ROW').getInfo())       
    #add的意思是将path1与row相加                  
    merge = path1.add(row)                
    #groupby的意思是将dartaframe根据条带来分组第0列为条带号）
    #print(merge)         
    a = merge.groupby([0])         
    #size为对groupby对象计算矩阵数量
    df1 = a.size()
    print(df1)
    
    #print(collection5.size().getInfo())
    #最后一步其实是没必要的，但是可以拿来检验
    

0
12139.0    16
12140.0    14
12239.0    14
12240.0    12
dtype: int64
0
12139.0    18
12140.0    12
12239.0    15
12240.0    11
dtype: int64
0
12139.0    17
12140.0    13
12239.0    11
12240.0    15
dtype: int64
0
12139.0    16
12140.0    14
12239.0    18
12240.0     8
dtype: int64
0
12139.0    15
12140.0    15
12239.0    19
12240.0     7
dtype: int64
0
12139.0    18
12140.0    12
12239.0    13
12240.0    13
dtype: int64
0
12139    16
12140    14
12239    14
12240    12
dtype: int64
0
12139.0    18
12140.0    12
12239.0     2
12240.0    11
dtype: int64
